# Dataset Prep

* Use *narrative* task data for content control
* For French data, remove participant 100 (outlier)
* is_L2 label: 1 = L2, 0 = L1 (native speaker)

## 1. Load Dataset

In [ ]:
import pandas as pd

file_path = 'your_path_here'

df = pd.read_csv(file_path)

In [ ]:
df

## 2. Select Required Rows

In [ ]:
all_narratives_df = df[df['group_task'].str.contains("narrative")] # remove all rows that aren't what we want
all_narratives_df = all_narratives_df.drop(all_narratives_df[all_narratives_df['participant_id'] == 100].index) # remove participant 100
all_narratives_df = all_narratives_df.reset_index(drop=True)
all_narratives_df

## 3. Add Labels

In [ ]:
input_df = all_narratives_df.copy()
input_df['is_L2'] = input_df['group_task'].apply(lambda x: 0 if 'nativespeaker' in x else 1)
input_df

In [ ]:
input_df = input_df.dropna(axis='columns') # remove columns with NaN (null) values

input_df

# Correlation Analysis

In [ ]:
ind_vars_dataset = input_df.drop(["group_task", "participant_id", "is_L2"], axis=1)  # all columns except target and identifying info (strings)
# correlation matrix
corr_matrix = ind_vars_dataset.corr()

In [ ]:
corr_matrix

## VIF Calculation

In [ ]:
# z-score normalization
means = ind_vars_dataset.mean()
std_devs = ind_vars_dataset.std()
z_scores = (ind_vars_dataset - means) / std_devs

In [ ]:
z_scores

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

vif_data = pd.DataFrame()
vif_data["feature"] = z_scores.columns

# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(z_scores.values, i)
                          for i in range(len(z_scores.columns))]

In [ ]:
vif_data

## Removing Highly Correlated Pairs

In [ ]:
import numpy as np

# find highly correlated pairs

abs_corr_matrix = corr_matrix.abs()
# upper triangle only
upper = abs_corr_matrix.where(np.triu(np.ones(abs_corr_matrix.shape), k=1).astype(bool))

high_corr_pairs = [(col, row, upper.loc[row, col])
                   for col in upper.columns
                   for row in upper.index
                   if upper.loc[row, col] > 0.8]

high_corr_pairs.sort(key=lambda x: x[2], reverse=True)

for col1, col2, corr in high_corr_pairs:
    print(f"{col1}, {col2}: {corr:.2f}")

In [ ]:
# discard the feature with a higher VIF

filtered_df = ind_vars_dataset

vif_data_indexed = vif_data.set_index('feature')

dropped_features = []

for col1, col2, corr in high_corr_pairs:
  print(f"comparing {col1} and {col2}")
  vif1 = vif_data_indexed.loc[col1, 'VIF']
  vif2 = vif_data_indexed.loc[col2, 'VIF']

  to_drop = col1 if vif1 >= vif2 else col2
  print(f"dropping {to_drop}")
  try:
    filtered_df = filtered_df.drop(columns=to_drop)
    dropped_features.append(to_drop)
  except:
    print(f"**already dropped {to_drop}**")

In [ ]:
print(f"\ntotal dropped features = {len(dropped_features)}")
print(f"features dropped: {dropped_features}")


total dropped features = 41
features dropped: ['BERT_stat_MeanK1', 'BERT_cuml_MeanK1', 'FT_Global', 'Sent_Global', 'BERT_cuml_ApEn', 'BERT_ApEn', 'FT_stat_Amp', 'BERT_stat_Acf', 'Sent_Amp', 'BERT_stat_Skew', 'BERT_cuml_Valley', 'BERT_stat_Valley', 'BERT_MeanK2', 'Sent_stat_WL', 'FT_WL', 'FT_cuml_ApEn', 'FT_stat_WL', 'Sent_stat_MeanK1', 'FT_Amp', 'FT_stat_MeanK1', 'Sent_cuml_MeanK1', 'BERT_cuml_WL', 'FT_cuml_WL', 'BERT_stat_MCR', 'BERT_cuml_Amp', 'BERT_stat_ApEn', 'Sent_Kurt', 'BERT_cuml_Kurt', 'BERT_stat_SSC', 'Sent_stat_Var', 'BERT_Amp', 'Sent_cuml_WL', 'BERT_stat_AcfZcr', 'Sent_cuml_Acf', 'BERT_cuml_Var', 'BERT_stat_Peak', 'BERT_Global', 'FT_cuml_Var', 'BERT_Acf', 'Sent_stat_Peak', 'BERT_cuml_Acf']


In [ ]:
filtered_df

In [ ]:
# update input df to only include desired features
input_df = input_df.drop(columns=dropped_features)
input_df

# Data Splitting

70% training, 30% testing

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = input_df.drop(["group_task", "participant_id", "is_L2"], axis=1)
y = input_df["is_L2"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Training samples: {len(X_train)}, Testing samples: {len(X_test)}")

# Build and Train Classifiers

## SVM Classifier

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC(probability=True, kernel='linear', random_state=42, class_weight ="balanced")
svm.fit(X_train, y_train)
svm

In [ ]:
svm_preds = svm.predict(X_test)
svm_preds

In [ ]:
svm_probs = svm.predict_proba(X_test)[:, 1]  # probability estimates for the positive class

## Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

In [ ]:
dtc = DecisionTreeClassifier(random_state=42, class_weight ="balanced")
dtc.fit(X_train, y_train)
dtc

In [ ]:
dtc_preds = dtc.predict(X_test)

In [ ]:
dtc_probs = dtc.predict_proba(X_test)[:, 1]

# Evaluate Models

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

## SVM

In [ ]:
svm_cm = confusion_matrix(y_test, svm_preds)
print("SVM Confusion Matrix:")
print(svm_cm)

SVM Confusion Matrix:
[[ 5  2]
 [18 34]]


In [ ]:
# heatmap
plt.figure(figsize=(5,4))
sns.heatmap(svm_cm, annot=True, fmt='d', cmap='Blues')
plt.title("SVM Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
print("SVM Classification Report:")
print(classification_report(y_test, svm_preds))

In [ ]:
# ROC Curve

svm_fpr, svm_tpr, svm_thresholds = roc_curve(y_test, svm_probs)
svm_auc = auc(svm_fpr, svm_tpr)

plt.figure(figsize=(6,5))
plt.plot(svm_fpr, svm_tpr, label=f"SVM ROC (AUC = {svm_auc:.2f})")
plt.plot([0,1], [0,1], 'k--', label="Random Chance")
plt.title("SVM ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()

## Decision Tree

In [ ]:
dtc_cm = confusion_matrix(y_test, dtc_preds)
print("Confusion Matrix:")
print(dtc_cm)

In [ ]:
# heatmap
plt.figure(figsize=(5,4))
sns.heatmap(dtc_cm, annot=True, fmt='d', cmap='Blues')
plt.title("DTC Confusion Matrix")
plt.xlabel("Predicted")
plt.ylabel("True")
plt.show()

In [ ]:
print("DTC Classification Report:")
print(classification_report(y_test, dtc_preds))

In [ ]:
# ROC Curve

dtc_fpr, dtc_tpr, dtc_thresholds = roc_curve(y_test, dtc_probs)
dtc_auc = auc(dtc_fpr, dtc_tpr)

plt.figure(figsize=(6,5))
plt.plot(dtc_fpr, dtc_tpr, label=f"DTC ROC (AUC = {dtc_auc:.2f})")
plt.plot([0,1], [0,1], 'k--', label="Random Chance")
plt.title("DTC ROC Curve")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.legend(loc="lower right")
plt.show()

# SHAP Values

## Utility Function

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
plt.style.use('default')

In [ ]:
!pip install shap

In [ ]:
import shap

In [ ]:
from scipy.special import softmax

def print_feature_importances_shap_values(shap_values, features):

    # feature importance (mean absolute shap value)
    importances = []
    for i in range(shap_values.values.shape[1]):
        importances.append(np.mean(np.abs(shap_values.values[:, i])))

    importances_norm = softmax(importances)

    feature_importances = {fea: imp for imp, fea in zip(importances, features)}
    feature_importances_norm = {fea: imp for imp, fea in zip(importances_norm, features)}

    # Sort dictionaries
    feature_importances = {k: v for k, v in sorted(feature_importances.items(), key=lambda item: item[1], reverse = True)}
    feature_importances_norm= {k: v for k, v in sorted(feature_importances_norm.items(), key=lambda item: item[1], reverse = True)}

    for k, v in feature_importances.items():
        print(f"{k} -> {v:.4f} (softmax = {feature_importances_norm[k]:.4f})")

## Calculate SHAP Values

### SVM

In [ ]:
# fit the explainer
svm_explainer = shap.Explainer(svm.predict, X_test)

In [ ]:
# calculates the SHAP values
svm_shap_values = svm_explainer(X_test)

In [ ]:
svm_shap_values

In [ ]:
shap.plots.bar(svm_shap_values)

In [ ]:
shap.summary_plot(svm_shap_values)

In [ ]:
print_feature_importances_shap_values(svm_shap_values, X_test.columns)

### DTC

In [ ]:
dtc_explainer = shap.Explainer(dtc.predict, X_test)

In [ ]:
dtc_shap_values = dtc_explainer(X_test)

In [ ]:
dtc_shap_values

In [ ]:
shap.plots.bar(dtc_shap_values)

In [ ]:
shap.summary_plot(dtc_shap_values)

In [ ]:
print_feature_importances_shap_values(dtc_shap_values, X_test.columns)